In [9]:
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092')
producer.send('test3', b'Hello, How are you!')
producer.flush()  # Ensure all messages are sent
producer.close()


In [10]:
from kafka import KafkaConsumer
import time

consumer = KafkaConsumer(
    'test3',
    group_id='test3-group',
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest'  # Add this line to start from the beginning of the topic
)


end_time = time.time() + 60  # for 1 minute
while time.time() < end_time:
    for message in consumer.poll(timeout_ms=1000).values():
        for msg in message:
            print(f"Received message: {msg.value.decode('utf-8')}")
            if time.time() > end_time:
                break
    if time.time() > end_time:
        break

consumer.close()


Received message: Hello, How are you!


In [11]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("KafkaIntegration") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()




In [17]:
# Define Kafka source for streaming queries
kafkaDataFrame = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "test3") \
    .option("startingOffsets", "earliest") \
    .option("failOnDataLoss", "false") \
    .load()



In [18]:
# Selecting the message value and converting it from binary to string
stringDF = kafkaDataFrame.selectExpr("CAST(value AS STRING)")

# Define the checkpoint location on HDFS
checkpoint_location = "hdfs://localhost:9000/user/priyasuresh/checkpoints"

# Define the output operation and start the query
query = stringDF.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("checkpointLocation", checkpoint_location) \
    .start()

query.awaitTermination()

24/02/29 14:10:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/02/29 14:10:55 WARN StreamingQueryManager: Stopping existing streaming query [id=eaee3f16-d1cc-4d02-9c9a-82cd64b3a745, runId=fd69ef4e-0b6d-450a-9806-31fbabcd7646], as a new run is being started.
24/02/29 14:10:55 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
24/02/29 14:10:55 WARN KafkaMicroBatchStream: Set(test1-0) are gone. Some data may have been missed.. 
Some data may have been lost because they are not available in Kafka any more; either the
 data was aged out by Kafka or the topic may have been deleted before all the data in the
 topic was processed. If you want your streaming query to fail on such cases, set the source
 option "failOnDataLoss" to "true".
    
24/02/29 14:10:55 WARN KafkaMicroBatchStream

-------------------------------------------
Batch: 6
-------------------------------------------
+-------------------+
|              value|
+-------------------+
|Hello, How are you!|
+-------------------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/priyasuresh/anaconda3/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/priyasuresh/anaconda3/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/priyasuresh/anaconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [21]:
query.stop()

In [22]:
spark.stop()

In [28]:
import json
import praw
from kafka import KafkaProducer
import re

# Function to clean text by removing URLs
def clean_text(text):
    return re.sub(r'http\S+', '', text)

# Reddit API credentials
client_id = 'v8QDtUYSpMtSphxDoCinVg'  # client_id
client_secret = '9ltLOwKtvcznBSenioCFdOpG9AhCog'  # client_secret
user_agent = 'For big data masters assignment to learn spark and kafka- SJSU MSDA'  #  user_agent
username = 'Studentgrad'
password = 'Welcome@123'

# Initialize PRAW with your credentials
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent,
    username=username,
    password=password
)

# Kafka Producer setup
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda m: json.dumps(m).encode('utf-8')
)

# Kafka topic name
topic_name = 'reddit'

# Specify the keyword for filtering
keyword = 'college'

# Stream posts from across all of Reddit
for submission in reddit.subreddit('all').stream.submissions():
    title_text = submission.title.lower()
    if keyword in title_text:
        title_clean = clean_text(submission.title)
        # Create a dictionary of the data you want to send
        data = {
            'title': title_clean,
            'selftext': clean_text(submission.selftext),
            'url': submission.url
        }
        # Send the preprocessed data to Kafka
        producer.send(topic_name, data)
        
        # Print out confirmation for debugging
        print(f"Sent to Kafka: {title_clean}")

# Make sure all messages are sent before closing
producer.flush()
producer.close()


ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]> returned error 61. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]> returned error 61. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]> returned error 61. Disconnecting.


Sent to Kafka: Anyone else have a lonely college experience?
Sent to Kafka: college discord servers
Sent to Kafka: Are you interested in buying? Hey my names Alo I sell pic 5$ video 10$ costume 15$ I’m a freshman in college and trying to pay bills
Sent to Kafka: The nine students at Alabama State College who were expelled in 1960 for their participation in a lunch counter sit-in. The students are: James McFadden, Joseph Peterson, St. John Dixon, Bernard Lee, Edward E. Jones, Leon Rice, Howard Shipman, Elroy Emory, and Marzette Watts
Sent to Kafka: the duality of discord servers in college
Sent to Kafka: Who here had a lonely college experience?
Sent to Kafka: University vs Liberal Arts College 
Sent to Kafka: Pronman: Top college, CHL and European free agents for NHL teams
Sent to Kafka: 24m straight college guy USA looking to chat


ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]> returned error 61. Disconnecting.


Sent to Kafka: Concorde college for RT program?
Sent to Kafka: I’m looking to get a new laptop for college. What is the best one I could get?
Sent to Kafka: Requesting for some Baruch College course syllabuses
Sent to Kafka: Looking for a college mentor from your dream school?
Sent to Kafka: Looking for a college mentor from your dream school? 
Sent to Kafka: Looking for a college mentor from your dream school? 
Sent to Kafka: 23 [M4F] Fit college stud looking to chat or sext pics with girls
Sent to Kafka: Tailgate for College night
Sent to Kafka: Looking for a college mentor from your dream school? 
Sent to Kafka: Alex leaving college
Sent to Kafka: If college education were free, would you go back to study? What would you study?
Sent to Kafka: Looking for a college mentor from your dream school? 
Sent to Kafka: Looking for a college mentor from your dream school? 
Sent to Kafka: COLLEGE POKER - CLUBGG
Sent to Kafka: Looking for a college mentor from your dream school? 


Sent to Kafka: Pardee (Boston U) or Marxe (Baruch College)
Sent to Kafka: 19 looking for a last minute college spring break cruise
Sent to Kafka: What is the criteria for international students to get accepted to an American college or university? Would they need to take IB or AP courses as prerequisites or are regular high school grades enough?


ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]> returned error 61. Disconnecting.


Sent to Kafka: Reddit, what is that one show about college students and milfs?
Sent to Kafka: College Advice
Sent to Kafka: Anyone have experience owning a OW as a college/university student?


ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]> returned error 61. Disconnecting.


Sent to Kafka: What are your biggest presentation fails in school/college/grad?
Sent to Kafka: i need advice as a college student
Sent to Kafka: Sino meron chinita/mabikini/college/jhs/shs/milf/petite? dm nyo ko tara
Sent to Kafka: Will Michigan fire Juwan Howard after a 30 point embarrassing loss to a 30th worst offense in College BB, Rutgers team (ranked 318 in ppg) that averages 64 ppg?
Sent to Kafka: i desperately need advice as a 19yo college student 
Sent to Kafka: My college now has a resident cat.
Sent to Kafka: Getting to know 5 graduate transfers finding success in college wrestling
Sent to Kafka: Trivia Nights &College Nights
Sent to Kafka: Nonprofit for mentoring College Seniors in Tech from NonTraditional Backgrounds: Looking for Mentors and Fellows
Sent to Kafka: College Town - Sandworm's Got a Salty Tooth (feat. Becky Feldman, Mel Cowan)
Sent to Kafka: Finds at the art building in the local community college
Sent to Kafka: College Mentorship Matching Program
Sent to Kafk

KeyboardInterrupt: 

In [29]:
producer.close()